#### Focal Loss

Focal Loss — это функция потерь, используемая в нейронных сетях для решения проблемы классификации *сложных* объектов. Идея состоит в том, что если мы имеем дело с сильным дисбалансом классов, то модели *просто* верно классифицировать объекты преобладаюшего класса (easy examples), а объекты минорного класса для нее являются *сложными* (hard examples). При этом, в силу дисбаланса, сумма большого количества малых потерь на *простых* объектах может перевешивать сумму малого количества больших потерь на *сложных* объектах, и тем самым модель будет плохо учиться верно классифицировать объекты минорного класса.

Focal Loss была предложена в статье [Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002) изначально для задачи детектирования объектов на изображениях. Она определяется так:

$$\text{FL}(p_t) = -(1 - p_t)^\gamma\text{log}(p_t)$$

Здесь $p_t$ — предсказанная вероятность истинного класса, а $\gamma$ — настраиваемый параметр. Focal Loss уменьшает потери на уверенно классифицируемых примерах (где $p_t>0.5$), и больше фокусируется на сложных примерах, которые классифицированы неправильно. Параметр $\gamma$ управляет относительной важностью неправильно классифицируемых примеров. Более высокое значение $\gamma$ увеличивает важность неправильно классифицированных примеров. В экспериментах авторы показали, что параметр $\gamma=2$ показывал себя наилучшим образом в их задаче.


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/focal_loss_vs_ce.png" width="700"></center>

<center><em>Source: <a href="https://arxiv.org/abs/1708.02002">Focal Loss for Dense Object Detection (Lin et al., 2018)</a></em></center>



При $\gamma=0$ Focal Loss становится равной Cross-Entropy Loss, которая выражается как обратный логарифм вероятности истинного класса:

$$\text{CE}(p_t)=-\text{log}(p_t)$$

Фактически, потери для уверенно классифицированных объектов дополнительно занижаются. Это похоже на взвешивание при дисбалансе классов.

Достигается этот эффект путем домножения на коэффициент: $ (1-p_{t})^\gamma$

Пока модель ошибается, $p_{t}$ — мала, и значение выражения в скобках соответственно близко к 1.

Когда модель обучилась, значение $p_{t}$ становится близким к 1, а разность в скобках становится маленьким числом, которое возводится в степень $ \gamma \ge 0 $. Таким образом, домножение на это небольшое число нивелирует вклад верно классифицированных объектов.

Это позволяет модели сосредоточиться (сфокусироваться, отсюда и название) на изучении сложных объектов (hard examples).





Разберем на примере. Пусть мы имеем дело с задачей бинарной классификации, где модель должна отличать яблоки и груши. Пусть набор данных несбалансирован: на 20 яблок приходится одна груша. Модель может хорошо обучиться классифицировать яблоки: вероятность истинного класса велика и равна $0.9$ для каждого яблока. При этом модель не научилась хорошо классифицировать груши: вероятность истинного класса для груши мала и равна $0.2$.

<img src='https://edunet.kea.su/repo/EduNet-content/L11/out/unbalanced_apples_pear.png' width=600></img>

$\large{CE = \overbrace{\sum^{20}-\text{log}(0.9)}^{\large\color{#3C8031}{\text{loss(apples)=2.11}}} + \overbrace{(-\text{log}(0.2))}^{\large\color{#F26035}{\text{loss(pear)=1.61}}} \approx 3.72}$

$\large{FL(\gamma=2) = \overbrace{\sum^{20}-\color{#AF3235}{\underbrace{(1-0.9)^2}_{0.01}}\text{log}(0.9)}^{\large\color{#3C8031}{\text{loss(apples)=0.02}}} + \overbrace{(-\color{#AF3235}{\underbrace{(1-0.2)^2}_{0.64}}\text{log}(0.2))}^{\large\color{#F26035}{\text{loss(pear)=1.03}}} \approx 1.05}$

В случае Focal Loss коэффициент $(1-p_t)^\gamma$ в 100 раз занизил потери при уверенной классификации яблок и потери при неверной классификации груши стали преобладать.

Давайте посчитаем для различных значений $γ$, сколько понадобится примеров с небольшой ошибкой (высокой вероятностью истинного класса, равной $0.9$), чтобы получить суммарный **Focal Loss** примерно такой же, как у одного примера с большой ошибкой (низкой вероятностью истинного класса, равной $0.2$).

In [ ]:
import numpy as np

def cross_entropy(prob_true):
    return -np.log(prob_true)


def focal_loss(prob_true, gamma=2):
    return (1 - prob_true) ** gamma * cross_entropy(prob_true)


p1 = 0.9  # probability of easy examples predictions
p2 = 0.2  # probability of hard examples predictions
gammas = [0, 0.5, 1, 2, 5, 10, 15]

print(
    f"For probability of easy examples predictions {p1} and probability of hard examples predictions {p2}\n"
)

for gamma in gammas:
    fl1 = focal_loss(p1, gamma)
    fl2 = focal_loss(p2, gamma)

    print(
        f"gamma = {gamma},".ljust(15),
        f"for an equal loss with a problematic prediction, almost correct ones are required {int(fl2 / fl1)}",
    )

Как видно, при увеличении значения $\gamma$ можно достичь значительного роста "важности" примеров с высокой ошибкой, что, по сути, позволяет модели обращать внимание на "hard examples".

В Focal Loss также могут быть добавлены веса для классов. Тогда формула будет выглядеть так:

$$\text{FL}(p_t) = -\alpha_t(1 - p_t)^\gamma\text{log}(p_t)$$

Здесь $\alpha_t$ — вес для истинного класса, имеющий такой же смысл, как параметр `weight` в Cross-Entropy Loss.

Focal Loss не реализована в PyTorch нативно, но существуют сторонние совместимые реализации. Посмотрим, как воспользоваться [одной из них](https://github.com/AdeelH/pytorch-multi-class-focal-loss).

In [ ]:
import random

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)

In [ ]:
from IPython.display import clear_output

!wget https://raw.githubusercontent.com/AdeelH/pytorch-multi-class-focal-loss/master/focal_loss.py
clear_output()

In [ ]:
import torch
from torch import nn
from focal_loss import FocalLoss


criterion = FocalLoss(alpha=None, gamma=2.)

model_output = torch.rand(3, 3)  # model output is logits, as in CELoss
print(f"model_output:\n {model_output}")

target = torch.empty(3, dtype=torch.long).random_(3)
print(f"target: {target}")

loss_fl = criterion(model_output, target)
print(f"loss_fl: {loss_fl}")

Убедимся, что сторонняя реализация вычисляет то, что нужно. Во-первых, переведем `model_output` из логитов в вероятности с помощью softmax.

In [ ]:
probs = torch.nn.functional.softmax(model_output, dim=1)

print(f"probabilities after softmax:\n {probs}")

In [ ]:
def cross_entropy(prob_true):
    return -np.log(prob_true)


def focal_loss(prob_true, gamma=2):
    return (1 - prob_true) ** gamma * cross_entropy(prob_true)

hand_calculated_loss = 0

for i in range(3):
    hand_calculated_loss += focal_loss(probs[i, target[i]])

hand_calculated_loss /= 3  # average by number of samples
print(f"hand-calculated focal loss: {hand_calculated_loss.item()}")